# Imports

In [114]:
from typing import List
import json 
import re
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
from pymystem3 import Mystem

[nltk_data] Downloading package stopwords to /home/diveev/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Data loading

In [115]:
data = pd.read_csv("../data/kp_films.csv")
data = data.dropna().reset_index().drop(columns='index')

In [116]:
data

,kp_id,rating_kp,genres,description
0,12819,7.70,['триллер' 'мелодрама' 'криминал'],"Миллиардер Томас Краун, пресыщенный финансист,..."
1,1260096,-1.00,['семейный' 'для' 'кино' 'ребенок'],Банда саранчи во главе с безжалостным Боссом о...
2,894673,4.40,['вестерн' 'иностранный' 'триллер' 'мистика' '...,"В горах живёт обособленная община, возглавляем..."
3,485364,6.06,['семейный' 'для' 'кино' 'короткометражный' 'р...,"Фильм высмеивает тех, кто понимает трудовую ди..."
4,1289632,6.40,['семейный' 'приключение' 'кино'],"В скучном городке Линдсборо всё меняется, когд..."
...,...,...,...,...
18494,806159,5.31,['семейный' 'аниме' 'приключение' 'фантастика'...,"Сатоши готовится к новому турниру, где парню н..."
18495,625381,6.40,['русский' 'мультфильм' 'для' 'ребенок' 'детск...,На нелепом и забавном мужчине по прозвищу Пара...
18496,4535755,-1.00,['семейный' 'мультфильм' 'иностранный' 'для' '...,Очаровательная сказка о маленьком лягушонке с ...
18497,1111361,6.60,['драма'],Криминальная драма о реальной истории жизни си...


# Data preprocessing

## *Preprocessing genres*

In [117]:
genres = data.genres.to_list()

for i in range(len(genres)):
    genres[i] = genres[i].replace('[', '')
    genres[i] = genres[i].replace(']', '')
    genres[i] = genres[i].replace(',', '')
    genres[i] = genres[i].replace("'", '')
    genres[i] = genres[i].split()

In [118]:
genres_corpus = []

for genres_list in genres:
    genres_corpus += genres_list

genres_corpus = set(genres_corpus)

In [119]:
genres_corpus = genres_corpus.difference(stopwords.words("russian")) #Removing stopwords from genres

In [120]:
for genre in genres_corpus:
    data[genre] = [1 if genre in genres[i] else 0 for i in range(data.shape[0])] #One hot encoding genres

In [121]:
data.head(10)

,kp_id,rating_kp,genres,description,романтика,аниме,ссср,шоу,сказка,мировой,...,триллер,спорт,ребенок,военный,мистика,детектив,вокруг,полнометражный,комедия,психология
0,12819,7.70,['триллер' 'мелодрама' 'криминал'],"Миллиардер Томас Краун, пресыщенный финансист,...",0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,1260096,-1.00,['семейный' 'для' 'кино' 'ребенок'],Банда саранчи во главе с безжалостным Боссом о...,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,894673,4.40,['вестерн' 'иностранный' 'триллер' 'мистика' '...,"В горах живёт обособленная община, возглавляем...",0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0
3,485364,6.06,['семейный' 'для' 'кино' 'короткометражный' 'р...,"Фильм высмеивает тех, кто понимает трудовую ди...",0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,1289632,6.40,['семейный' 'приключение' 'кино'],"В скучном городке Линдсборо всё меняется, когд...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,960385,6.30,['комедия'],С самого начала отношений страстные любовники ...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
6,43127,7.30,['драма' 'советский' 'военный' 'ссср'],"Советский художественный фильм, рассказывающий...",0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
7,331881,5.30,['ссср' 'советский' 'комедия'],"Если привычная жизнь уже порядком надоела, а м...",0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
8,259905,-1.00,['русский' 'советский' 'мультфильм' 'для' 'реб...,Очаровательный щенок Боб больше всего любит сл...,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
9,487544,6.83,['русский' 'семейный' 'документальный' 'познав...,"Режиссер Валерий Бабич, автор исторических про...",0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


## *Filtering genres*

In [122]:
#Drop the least appearing genres

genre_to_num = {genre : (data[genre] != 0).sum() for genre in genres_corpus if (data[genre] != 0).sum() > 50}
one_hot_encoded_genres = data[list(genre_to_num.keys())]
descriptions = data[~(one_hot_encoded_genres==0).all(axis=1)].description
one_hot_encoded_genres = one_hot_encoded_genres.loc[~(one_hot_encoded_genres==0).all(axis=1)].reset_index().drop(columns='index')

In [123]:
one_hot_encoded_genres.head(5)

,аниме,ссср,шоу,сказка,мировой,мультфильм,семейный,тв,драма,хотеть,...,музыкальный,криминал,триллер,спорт,ребенок,военный,мистика,детектив,полнометражный,комедия
0,0,0,0,0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,0,0
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
3,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## *Preprocessing and encoding descriptions*

In [124]:
def normalize_text(strings:List[str]) -> List[str]:
    normalized = []
    
    for string in strings:
        string = re.findall(r"\w+", str(string))
        string = " ".join(string)
        string = re.sub(r"\s+", " ", string)
        string = string.lower()
        
        normalized.append(string)
        
    return normalized

In [125]:
def filter_stopwords(strings:List[str]) -> List[str]:
    filtered = []
    
    for string in strings:
        words = string.split()
        words = [w for w in words if w not in stopwords.words("russian")]
        string = " ".join(words)
        
        filtered.append(string)
        
    return filtered

In [126]:
def lemmatize_words(strings:List[str]) -> List[str]:
    lemmatized = []
    stem = Mystem()
    
    for string in strings:
        lemmas = stem.lemmatize(string)
        lemmas[-1] = lemmas[-1].replace("\n", "")
        string = ''.join(lemmas)
        
        lemmatized.append(string)
        
    return lemmatized

In [127]:
def preprocessing_pipeline(strings:List[str]) -> List[str]:
    strings = normalize_text(strings)
    strings = filter_stopwords(strings)
    strings = lemmatize_words(strings)
    
    return strings

In [ ]:
corpus = preprocessing_pipeline(descriptions)

In [ ]:
id_to_text = {i : corpus[i] for i in range(len(corpus))}

In [ ]:
with open("../data/movie_descriptions.json", "w") as f:
    f.write(json.dumps(id_to_text))

In [ ]:
with open("../data/movie_descriptions.json") as f:
    data = json.loads(f.read())
descriptions = list(data.values())

In [ ]:
vectorizer = TfidfVectorizer()
corpus = vectorizer.fit_transform(descriptions)

In [ ]:
corpus

## *Creating labels for descriptions*

In [ ]:
labels = one_hot_encoded_genres.apply(np.flatnonzero, axis=1)

In [ ]:
labels